In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField, TimestampType, IntegerType
from pyspark.sql.functions import from_json, col

# Define the path to the jars on the EC2 instance
spark_jars_path = "/home/ec2-user/stream-processing-template/jars"  # <-- Update this path

spark = SparkSession.builder.appName("retail_pysaprk_consumer") \
    .config("spark.jars", f"{spark_jars_path}/commons-pool2-2.11.1.jar,"
            f"{spark_jars_path}/spark-sql-kafka-0-10_2.12-3.4.0.jar,"
            f"{spark_jars_path}/spark-streaming-kafka-0-10-assembly_2.12-3.4.0.jar") \
    .getOrCreate()


23/12/13 10:28:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/usr/local/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [2]:
# Define the schema for our data
schema = StructType([
    StructField("store_location", StringType(), True),
    StructField("time_of_purchase", TimestampType(), True),
    StructField("product_ID", StringType(), True),
    StructField("transaction_amount", IntegerType(), True)
])

# Stream from Kafka topic
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "b-1.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092") \
    .option("subscribe", "retail_transactions") \
    .load()

In [3]:
transactions = (df.selectExpr("CAST(value AS STRING)")
                .withColumn("data", from_json(col("value"), schema))
                .select("data.*"))

query = transactions.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

23/12/13 10:28:47 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bea4f1cc-4f28-46fd-8e12-e586f554b3f2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/13 10:28:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/12/13 10:28:49 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+----------------+----------+------------------+
|store_location|time_of_purchase|product_ID|transaction_amount|
+--------------+----------------+----------+------------------+
+--------------+----------------+----------+------------------+



-------------------------------------------
Batch: 1
-------------------------------------------


+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|  Philadelphia|2023-12-13 10:28:52|    P00033|               431|
+--------------+-------------------+----------+------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|       Chicago|2023-12-13 10:28:54|    P00031|               428|
|   Los Angeles|2023-12-13 10:28:55|    P00069|               274|
+--------------+-------------------+----------+------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|       Houston|2023-12-13 10:28:57|    P00078|               461|
+--------------+-------------------+----------+------------------+



...

-------------------------------------------
Batch: 175
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|       Chicago|2023-12-13 10:34:47|    P00039|               240|
+--------------+-------------------+----------+------------------+

-------------------------------------------
Batch: 176
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|  Philadelphia|2023-12-13 10:34:49|    P00044|               118|
+--------------+-------------------+----------+------------------+

-------------------------------------------
Batch: 177
-------------------------------------------
+--------------+---------------

-------------------------------------------
Batch: 178
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|      New York|2023-12-13 10:34:52|    P00075|                61|
+--------------+-------------------+----------+------------------+

-------------------------------------------
Batch: 179
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|      New York|2023-12-13 10:34:55|    P00022|               778|
+--------------+-------------------+----------+------------------+

-------------------------------------------
Batch: 180
-------------------------------------------
+--------------+---------------

-------------------------------------------
Batch: 192
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|   Los Angeles|2023-12-13 10:35:22|    P00044|               908|
+--------------+-------------------+----------+------------------+



-------------------------------------------
Batch: 193
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|       Chicago|2023-12-13 10:35:23|    P00005|               227|
+--------------+-------------------+----------+------------------+

-------------------------------------------
Batch: 194
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|  Philadelphia|2023-12-13 10:35:24|    P00082|               612|
+--------------+-------------------+----------+------------------+

-------------------------------------------
Batch: 195
-------------------------------------------
+--------------+---------------

-------------------------------------------
Batch: 204
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|  Philadelphia|2023-12-13 10:35:44|    P00093|                24|
+--------------+-------------------+----------+------------------+

